In [1]:
%pip install ccxt

In [2]:
import ccxt
import time
import numpy as np

class HFTStrategy:
    def __init__(self, exchange_id, api_key, secret):
        self.exchange = getattr(ccxt, exchange_id)({
            'apiKey': api_key,
            'secret': secret,
            'enableRateLimit': True,
        })

    def fetch_order_book(self, symbol):
        return self.exchange.fetch_order_book(symbol)

    def place_order(self, symbol, order_type, side, amount, price=None):
        if order_type == 'limit':
            return self.exchange.create_order(symbol, order_type, side, amount, price)
        elif order_type == 'market':
            return self.exchange.create_order(symbol, order_type, side, amount)

    def run(self, symbol, amount):
        while True:
            order_book = self.fetch_order_book(symbol)
            bid = order_book['bids'][0][0] if len(order_book['bids']) > 0 else None
            ask = order_book['asks'][0][0] if len(order_book['asks']) > 0 else None
            if bid and ask:
                spread = ask - bid
                if spread > 0:
                    self.place_order(symbol, 'limit', 'buy', amount, bid)
                    self.place_order(symbol, 'limit', 'sell', amount, ask)
            time.sleep(0.1)

if __name__ == "__main__":
    api_key = 'your_api_key'
    secret = 'your_secret'
    symbol = 'BTC/USDT'
    amount = 0.001
    hft_strategy = HFTStrategy('binance', api_key, secret)
    hft_strategy.run(symbol, amount)